This notebook used the list of verified IUCN codes mapped to GAP species codes provided by Leah Dunn and added them to the identifiers for the 1533 species items in both the Habitat Maps and Range Maps collections. It doesn't really need to be touched again unless something changes.

In [43]:
import pandas as pd
from IPython.display import display
import pysb
import time
import sys
import requests
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [44]:
sb = pysb.SbSession()
username = input("Username: ")
sb.loginc(str(username))

Username: sbristol@usgs.gov
········


In [46]:
habitatMapCollection = sb.get_item("527d0a83e4b0850ea0518326")

In [48]:
for file in habitatMapCollection["files"]:
    if file["name"] == "IUCN_Gap.csv":
        iucnGap = pd.read_csv(StringIO(sb.get(file["url"])))

In [49]:
display (iucnGap)

,ComName,SciName,Taxa,GapSpCode,IUCN Spp Number,Authority
0,Abert's Squirrel,Sciurus aberti,M,mABSQx,42461,"Woodhouse, 1853"
1,Abert's Towhee,Pipilo aberti,B,bABTOx,22721315,"(Baird, 1852)"
2,Acadian Flycatcher,Empidonax virescens,B,bACFLx,22699842,"(Vieillot, 1818)"
3,Acorn Woodpecker,Melanerpes formicivorus,B,bACWOx,22680813,"(Swainson, 1827)"
4,Agile Kangaroo Rat,Dipodomys agilis,M,mAKRAx,6684,"Gambel, 1848"
5,Alabama Map Turtle,Graptemys pulchra,R,rALTUx,170494,"Baur, 1893"
6,Alabama Red-bellied Cooter,Pseudemys alabamensis,R,rARBCx,18458,"Baur, 1893"
7,Alder Flycatcher,Empidonax alnorum,B,bALFLx,22699845,"Brewster, 1895"
8,Allegheny Mountain Dusky Salamander,Desmognathus ochrophaeus,A,aAMDSx,59253,"Cope, 1859"
9,Allegheny Woodrat,Neotoma magister,M,mALWOx,14581,"Baird, 1858"


In [50]:
count = 0
for index, row in iucnGap.iterrows():
    sbItems = sb.find_items("parentId=527d0a83e4b0850ea0518326&fields=identifiers&q="+row["GapSpCode"])
    updateItem = {}
    updateItem["id"] = sbItems["items"][0]["id"]
    updateItem["identifiers"] = []
    for identifier in sbItems["items"][0]["identifiers"]:
        identifierTerm = requests.get(identifier["scheme"].replace("/category/","/term/")+"?format=json").json()
        _thisIdentifier = {}
        _thisIdentifier["key"] = identifier["key"]
        _thisIdentifier["type"] = identifierTerm["name"]
        _thisIdentifier["scheme"] = identifierTerm["scheme"]+"/"+identifierTerm["name"]
        updateItem["identifiers"].append(_thisIdentifier)

    _iucnIdentifier = {}
    _iucnIdentifier["type"] = "iucn_id_verified"
    _iucnIdentifier["scheme"] = "https://www.sciencebase.gov/vocab/category/bis/bis_identifiers/term/iucn_id_verified"
    _iucnIdentifier["key"] = row["IUCN Spp Number"]
    updateItem["identifiers"].append(_iucnIdentifier)
    count = count + 1
    sb.update_item(updateItem)
    print (updateItem["id"], count)

58fa5fc3e4b0b7ea545256b1 1
58fa4438e4b0b7ea54524c22 2
58fa443ee4b0b7ea54524c25 3
58fa4444e4b0b7ea54524c29 4
58fa6037e4b0b7ea545256d2 5
58fe0a55e4b0074928294569 6
58fe0ab2e4b007492829457b 7
58fa444ae4b0b7ea54524c2c 8
58fa3f0ee4b0b7ea5452485c 9
58fa605fe4b0b7ea545256dd 10
58fa5f9ee4b0b7ea545256a8 11
58fa4451e4b0b7ea54524c30 12
58fe0ac9e4b0074928294580 13
58fa604ce4b0b7ea545256d7 14
58fa4458e4b0b7ea54524c33 15
58fa3f09e4b0b7ea54524856 16
58fe0a6be4b007492829456e 17
58fa445ee4b0b7ea54524c37 18
58fa6073e4b0b7ea545256e2 19
58fa6084e4b0b7ea545256e6 20
58fa620be4b0b7ea54525758 21
58fa4463e4b0b7ea54524c3a 22
58fa5f8be4b0b7ea545256a2 23
58fa4431e4b0b7ea54524c1f 24
58fa3f0be4b0b7ea54524859 25
58fa4469e4b0b7ea54524c3e 26
58fe0ffae4b007492829464a 27
58fa446ee4b0b7ea54524c41 28
58fa4475e4b0b7ea54524c44 29
58fa447be4b0b7ea54524c48 30
58fa4481e4b0b7ea54524c4b 31
58fa6022e4b0b7ea545256cd 32
58fa4487e4b0b7ea54524c50 33
58fa6095e4b0b7ea545256eb 34
58fa60a6e4b0b7ea545256f1 35
58fa448de4b0b7ea54524c53 36
5